In [65]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # or "0,1"

In [ ]:
wandb.init(project="tesi", name="bigbird")  # args = TrainingArguments(
wandb.stop()

In [1]:
import json
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from datasets import Dataset

tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large")
model = AutoModelForQuestionAnswering.from_pretrained(
    "facebook/bart-large", return_dict=False
).to("cuda:0")

/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.0.4) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
Some weights of BartForQuestionAnswering were not initialized from the model checkpoint at facebook/bart-large and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
import json
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

tokenized = []
file_path = "./nil_el_instanceof.jsonl"
from datasets import Dataset

original_data = []
with open(file_path, "r") as file:
    for line in file:
        # Load each line as a JSON object
        data_line = json.loads(line)
        
        
        original_data.append(data_line)
print(len(original_data))

7939


In [3]:
import json
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

tokenized = []
file_path = "./aida_train_instanceof.jsonl"
from datasets import Dataset

original_data_aida = []
with open(file_path, "r") as file:
    for line in file:
        # Load each line as a JSON object
        data_line = json.loads(line)
        
        # additional_question = "<additional> "
        # try:
        #     for related in data_line["most_related"]:
        #         length = len(related)
        #         if len(related) == 4:
        #             additional_question += related[2]
        #             additional_question += " "
        # except:
        #     None
        # additional_question += "</additional>"
        # data_line["context"] += additional_question
        original_data_aida.append(data_line)
print(len(original_data_aida))

18448


In [4]:
import random
random.shuffle(original_data_aida)
#concat = original_data_aida
concat =original_data_aida[:int(len(original_data_aida)* 0.4)] + original_data


random.shuffle(concat)
#print(len(concat))
train_original = concat[: int(len(concat) * 0.95)]
eval_original = concat[int(len(concat) * 0.95) : ]
dataset = Dataset.from_dict(
    {
        "context": [item["question"] for item in train_original],
        "question": [item["context"] for item in train_original],
        "answers": [item["answers"] for item in train_original],
    }
)
dataset_ev = Dataset.from_dict(
    {
        "context": [item["question"] for item in eval_original],
        "question": [item["context"] for item in eval_original],
        "answers": [item["answers"] for item in eval_original],
    }
)

In [8]:
def get_correct_alignement(context, answer):
    """Some original examples in SQuAD have indices wrong by 1 or 2 character. We test and fix this here."""
    gold_text = answer["text"][0]
    
    start_idx = context.find(gold_text)
    end_idx = context.find("</ec>", start_idx)
    return start_idx, end_idx
    # if context[start_idx:end_idx] == gold_text:
    #     return start_idx, end_idx       # When the gold label position is good
    # elif context[start_idx-1:end_idx-1] == gold_text:
    #     return start_idx-1, end_idx-1   # When the gold label is off by one character
    # elif context[start_idx-2:end_idx-2] == gold_text:
    #     return start_idx-2, end_idx-2   # When the gold label is off by two character
    # else:
    #     raise ValueError()


# Tokenize our training dataset
def convert_to_features(example):
    try:
        # Tokenize contexts and questions (as pairs of inputs)

        input_pairs = [example["question"], example["context"]]
        encodings = tokenizer.encode_plus(
            input_pairs,max_length=512, pad_to_max_length=False, truncation="only_second"
        )
        context_encodings = tokenizer.encode_plus(example["context"])

        # Compute start and end tokens for labels using Transformers's fast tokenizers alignement methodes.
        # this will give us the position of answer span in the context text

        start_idx, end_idx = get_correct_alignement(
            example["context"], example["answers"]
        )
        if end_idx != -1 and start_idx != -1:
            try:
                start_positions_context = context_encodings.char_to_token(start_idx)
                end_positions_context = context_encodings.char_to_token(end_idx)

                # here we will compute the start and end position of the answer in the whole example
                # as the example is encoded like this <s> question</s></s> context</s>
                # and we know the postion of the answer in the context
                # we can just find out the index of the sep token and then add that to position + 1 (+1 because there are two sep tokens)
                # this will give us the position of the answer span in whole example
                sep_idx = encodings["input_ids"].index(tokenizer.sep_token_id)
                start_positions = start_positions_context + sep_idx
                end_positions = end_positions_context + sep_idx + 1
                #print(tokenizer.decode
                #(encodings["input_ids"][start_positions:end_positions]))
                encodings.update(
                    {
                        "start_positions": start_positions,
                        "end_positions": end_positions,
                        "attention_mask": encodings["attention_mask"],
                    }
                )
                return encodings
            except Exception as e:
                print(e)
        else:
            print("qu")
            encodings.update(
                {
                    "start_positions": start_idx,
                    "end_positions": end_idx,
                    "attention_mask": encodings["attention_mask"],
                }
            )
            return encodings
    except Exception as e:
        print("errors", e)

In [9]:
train_ds = dataset.map(convert_to_features)
eval_ds = dataset_ev.map(convert_to_features) 

Map:   0%|          | 0/14552 [00:00<?, ? examples/s]

qu
qu
qu
qu
qu
qu
qu
qu
qu
qu
qu
qu
qu
qu
qu
qu
qu
qu
qu
qu
qu


Map:   0%|          | 0/766 [00:00<?, ? examples/s]

qu


In [ ]:
import joblib
import datasets

In [ ]:
joblib.dump( train_ds, './dumps/aidaTrainIOF.dump',)
joblib.dump( eval_ds, './dumps/aidaEvalIOF.dump',)

In [ ]:
train_ds = joblib.load('./dumps/aidaTrainNERBigBird.dump')
eval_ds = joblib.load('./dumps/aidaEvalNERBigBird.dump')

In [ ]:
wandb.init()

In [10]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
from transformers import default_data_collator

import wandb
torch.cuda.set_device(0)
torch.cuda.current_device()

#wandb.init(project="tesi", name="robertaAidaLargeNERNIL")  # args = TrainingArguments(
#     f"longformer-finetuned-squad",
#     evaluation_strategy="epoch",
#     learning_rate=2e-5,
#     per_device_train_batch_size=8,
#     num_train_epochs=3,
#     weight_decay=0.01,
# )

args = TrainingArguments(
    "bart",
    evaluation_strategy="steps",
    save_strategy="steps",
    report_to="wandb",
    load_best_model_at_end=False,
    logging_strategy="steps",
    logging_steps=100,
    eval_steps=100,
    save_steps=100,
    save_total_limit=1,
    learning_rate=3e-5,
    per_device_eval_batch_size=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=10,
    num_train_epochs=1,
    weight_decay=0.02,
    fp16=True,
)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    tokenizer=tokenizer,
)

In [11]:
trainer.train()

/home/rub/.local/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.0.4) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
wandb: Currently logged in as: agazzi-ruben99. Use `wandb login --relogin` to force relogin


  0%|          | 0/1455 [00:00<?, ?it/s]

You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 1.8309, 'learning_rate': 2.802061855670103e-05, 'epoch': 0.07}


  0%|          | 0/766 [00:00<?, ?it/s]

...

  0%|          | 0/766 [00:00<?, ?it/s]

{'eval_loss': nan, 'eval_runtime': 26.5478, 'eval_samples_per_second': 28.854, 'eval_steps_per_second': 28.854, 'epoch': 0.89}
{'loss': 0.4573, 'learning_rate': 1.2371134020618557e-06, 'epoch': 0.96}


  0%|          | 0/766 [00:00<?, ?it/s]

{'eval_loss': nan, 'eval_runtime': 26.5341, 'eval_samples_per_second': 28.868, 'eval_steps_per_second': 28.868, 'epoch': 0.96}
{'train_runtime': 2408.7444, 'train_samples_per_second': 6.041, 'train_steps_per_second': 0.604, 'train_loss': 0.7171860921014216, 'epoch': 1.0}


TrainOutput(global_step=1455, training_loss=0.7171860921014216, metrics={'train_runtime': 2408.7444, 'train_samples_per_second': 6.041, 'train_steps_per_second': 0.604, 'train_loss': 0.7171860921014216, 'epoch': 1.0})

In [ ]:
trainer.save_model('./RoBertAAida')

In [ ]:
torch.cuda.empty_cache()

In [ ]:
dataset[0]

## Prediction and evaluation

In [13]:
def check_brackets(input_string):
    stack = []
    brackets = {"(": ")", "{": "}", "[": "]"}
    for char in input_string:
        if char in brackets.keys():
            stack.append(char)
        elif char in brackets.values():
            if not stack or brackets[stack.pop()] != char:
                return False
    return not stack


def process_answer(answer, candidates):
    if answer == "Not In Candidates":
        return answer
    else:
        modified_answer = answer.split("</ec>")[0]
        modified_answer = modified_answer.split(": instance of ")[0]
        modified_answer = modified_answer.split(": instance")[0]
        modified_answer = modified_answer.replace("<s>", "")
        modified_answer = modified_answer.replace("</ec", "")
        modified_answer = modified_answer.replace("</s>", "")
        modified_answer = modified_answer.replace("<s", "")
        modified_answer = modified_answer.replace("</", "")
        modified_answer = modified_answer.replace(" ec", "")
        modified_answer = modified_answer.replace(">", "")
        modified_answer = modified_answer.strip()
        if not check_brackets(modified_answer):
            modified_answer = modified_answer.replace("(", "")
            modified_answer = modified_answer.replace(")", "")
        if modified_answer == "Not In Candidates":
            modified_answer = "NIL"
        if modified_answer == "":
            modified_answer = "Not In Candidates"
        if modified_answer not in candidates:
            modified_answer = "Not In Candidates"
        return modified_answer


process_answer("> Alzenau </", ["Alzenau"])

'Alzenau'

In [36]:
import gc
import torch.nn.functional as F


def make_prediction(data_entry, nil_prediction):
    with torch.no_grad():
        question = data_entry["input"]
        context = ""
        # if nil_prediction:
        #     context += " Not In Candidates </ec> "
        index = 0
        added = False
        for item in data_entry["candidates"]:
            context += item + f" </ec> "
            index += 1
            if index == 1 and nil_prediction:
                context += " Not In Candidates : instance of Unknown </ec> "
                added = True
        if not added and nil_prediction:
            context = " Not In Candidates : instance of Unknown </ec> " + context
        input_pairs = [question, context]

        encodings = tokenizer.encode_plus(
            input_pairs, return_tensors="pt", truncation="only_second"
        ).to("cuda")
        start_scores, end_scores = model(
            encodings["input_ids"], attention_mask=encodings["attention_mask"]
        )
        start_scores = F.softmax(start_scores, dim=1)
        end_scores = F.softmax(end_scores, dim=1)
        # start_scores.to("cpu")
        # end_scores.to("cpu")
        start = torch.argmax(
            start_scores
        )  # Get the most likely beginning of answer with the argmax of the score
        end = (
            torch.argmax(end_scores) + 1
        )  # Get the most likely end of answer with the argmax of the score
        mean_score = 0
        try:
            mean_score = end_scores[0][end.item()] + start_scores[0][start.item()]
        except:
            None
        answer_tokens = encodings["input_ids"][0, start.item() : end.item() + 1]
        answer = ""

        answer = process_answer(tokenizer.decode(answer_tokens), context)
        classified = 0
        if (mean_score < 0.9) and nil_prediction:
            answer = "Not In Candidates"
        else:
            if answer == "":
                answer = "Not In Candidates"
        del encodings
        del end_scores
        del start_scores
        del start
        del end
        score = float(mean_score)
        gc.collect()
        torch.cuda.empty_cache()
        return {
            "correct": data_entry["output"],
            "non_processed": tokenizer.decode(answer_tokens),
            "predicted": answer,
            "input_phrase": question,
            "scores": score,
            "candidates": context,
        }

In [ ]:
import wandb
wandb.init()

In [37]:
from tqdm import tqdm
import json
import torch

results = []
file_path = "./Datasets/InstanceOf/msnbc_test_instanceof-nil.jsonl"
dataset = []
with open(file_path, "r") as file:
    for line in file:
        # Load each line as a JSON object
        data_line = json.loads(line)

        dataset.append(data_line)

for item in tqdm(dataset):
    try:
        pred = make_prediction(item, True)
        results.append(pred)
    except Exception as e:
        print(e)

  0%|          | 0/656 [00:00<?, ?it/s]

100%|██████████| 656/656 [01:23<00:00,  7.83it/s]


In [31]:
correct

549

In [38]:
correct = 0
correct_trace = []
wrong = []
correct_nil = 0
wrong_nil = 0
full_data = []
for result in tqdm(results):
  processed = process_answer(result['non_processed'], result['candidates'])
  if processed == result['correct'][0]['answer']:
    correct += 1
    correct_trace.append(result)
    if processed == 'Not In Candidates':
        correct_nil += 1
  else:
    if result['correct'][0]['answer'] == 'Not In Candidates':
        wrong_nil += 1
    wrong.append(result)
  full_data.append({'score': result['scores'], 'nil': int(result['correct'][0]['answer'] == 'Not In Candidates')} )

100%|██████████| 656/656 [00:00<00:00, 183504.30it/s]


In [39]:
print(f"accuracy of the model is {correct/len(results)}")
print(f"accuracy in nil prediction is: {correct_nil/(correct_nil+wrong_nil)}")

accuracy of the model is 0.8414634146341463
accuracy in nil prediction is: 0.6903225806451613


In [33]:
print(f"accuracy of the model is {correct/len(results)}")
print(f"accuracy in nil prediction is: {correct_nil/(correct_nil+wrong_nil)}")

accuracy of the model is 0.836890243902439
accuracy in nil prediction is: 0.5935483870967742


-  Base: 65.9%
-  Large: 73.21%(enriched) - 74.65% (standard)
-  Large EN: 76.66%
-  large NER: 75.5%
-  Base-NER: 66.35%

## MSNBC large
-  No EN: 91.92%
-  No Instance Of: 92.83% - Yes: 93.44%
-  EN: 92.8
-  Large NER: 92.83%

In [ ]:
len(wrong)

In [34]:
import pandas as pd
full_df = pd.DataFrame(full_data)
df = pd.DataFrame(wrong)
df_c = pd.DataFrame(correct_trace)

In [115]:
df.iloc[0]['candidates']

'Iraq War : instance of military offensive </ec> Iraq national football team : instance of national association football team </ec> Anglo-Iraqi War : instance of war </ec> Iraq Football Association : instance of association football federation </ec> Iraqi Army : instance of Unknown </ec> Iraq at the 2004 Summer Olympics : instance of Olympic delegation </ec> Iraq at the 2008 Summer Olympics : instance of Olympic delegation </ec> Iraq at the 1960 Summer Olympics : instance of Olympic delegation </ec> Iraq prison abuse scandals : instance of scandal </ec> Iraq at the Asian Games : instance of nation at sport competition </ec> Iraq national basketball team : instance of Unknown </ec> History of Iraq (2003–11) : instance of Unknown </ec> Iraq at the 2006 Asian Games : instance of nation at sport competition </ec> Iraqi Premier League : instance of Unknown </ec> Iraq national futsal team : instance of national futsal team </ec> Afro-Iraqi : instance of Unknown </ec> Iraq–Turkey relations : 

In [35]:
n = 20
offset=1
df.iloc[offset *n : offset*n + n]

,correct,non_processed,predicted,input_phrase,scores,candidates
20,"[{'answer': 'Not In Candidates', 'provenance':...","</s>Tuscaloosa County, Alabama : instance of c...","Tuscaloosa County, Alabama",[START_ENT] Tuscaloosa [END_ENT] Ala Nick Saba...,0.999993,"Tuscaloosa County, Alabama : instance of count..."
21,"[{'answer': 'University of Alabama', 'provenan...",</s>Alabama : instance of U.S. state </ec,Alabama,TUSCALOOSA Ala Nick Saban couldn t escape his ...,0.957006,Alabama : instance of U.S. state </ec> Univers...
22,"[{'answer': 'Not In Candidates', 'provenance':...",</s>Miami : instance of city in the United Sta...,Miami,TUSCALOOSA Ala Nick Saban couldn t escape his ...,0.994787,Miami : instance of city in the United States ...
23,"[{'answer': 'Alabama Crimson Tide', 'provenanc...",> Alabama Crimson Tide football : instance of ...,Alabama Crimson Tide football,TUSCALOOSA Ala Nick Saban couldn t escape his ...,0.999967,Crimson Tide (film) : instance of film </ec> A...
24,"[{'answer': 'Alabama', 'provenance': [{'title'...",> Alabama Crimson Tide football : instance of ...,Alabama Crimson Tide football,to leave the Miami Dolphins In other words he ...,0.981053,Alabama : instance of U.S. state </ec> Univers...
25,"[{'answer': 'Alabama Crimson Tide', 'provenanc...",> Alabama Crimson Tide football : instance of ...,Alabama Crimson Tide football,s here I love it here he said I like to affect...,0.999995,Tide : instance of Unknown </ec> Tide (brand) ...
26,"[{'answer': 'University of Alabama', 'provenan...",> Alabama Crimson Tide football : instance of ...,Alabama Crimson Tide football,in the past It s what you do now he said His r...,0.999062,Alabama : instance of U.S. state </ec> Univers...
27,"[{'answer': 'Louisiana State University', 'pro...",> LSU Tigers football : instance of college sp...,LSU Tigers football,It s what you do now he said His resume featur...,0.999994,Louisiana State University : instance of publi...
28,"[{'answer': 'Not In Candidates', 'provenance':...",</s>Miami : instance of city in the United Sta...,Miami,Alabama fans hunger for most A national title ...,0.999993,Miami : instance of city in the United States ...
29,"[{'answer': 'University of Alabama', 'provenan...",> Alabama Crimson Tide football : instance of ...,Alabama Crimson Tide football,fans hunger for most A national title His cham...,0.999566,Alabama : instance of U.S. state </ec> Univers...


In [44]:
df.to_csv("./errorsNil.csv")
df_c.to_csv("./correctNil.csv")